In [1]:
from datasets import load_dataset
dataset = load_dataset("locuslab/TOFU","full")

Using the latest cached version of the dataset since locuslab/TOFU couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'full' at /data/.cache/huggingface/datasets/locuslab___tofu/full/0.0.0/c3ad0b4efbdd12e11cc4faf95aebfbd6f53b664b (last modified on Thu Oct 24 14:46:23 2024).


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 4000
    })
})

In [3]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os

In [93]:
#,
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5",trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [63]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


In [81]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    trust_remote_code=True,
    use_flash_attention_2 = True ,torch_dtype = torch.bfloat16 
)

In [13]:
def find_target_modules(model):
    # Initialize a Set to Store Unique Layers
    unique_layers = set()
    
    # Iterate Over All Named Modules in the Model
    for name, module in model.named_modules():
        # Check if the Module Type Contains 'Linear8bitLt' or 'Linear4bit'
        module_type = str(type(module))
        if "Linear8bitLt" in module_type or "Linear4bit" in module_type:
            # Extract the Type of the Layer
            layer_type = name.split('.')[-1]
            
            # Add the Layer Type to the Set of Unique Layers
            unique_layers.add(layer_type)

    # Return the Set of Unique Layers Converted to a List
    return list(unique_layers)

target_modules = find_target_modules(model)
target_modules

['v_proj', 'q_proj', 'k_proj', 'dense', 'fc2', 'fc1']

In [14]:
def find_all_linear_names(model):
    cls = torch.nn.Linear
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])
    if 'lm_head' in lora_module_names: # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)
k = find_all_linear_names(model)
k

['v_proj', 'q_proj', 'k_proj', 'dense', 'fc2', 'fc1']

In [16]:
config = LoraConfig(
            r= 8, 
            lora_alpha= 32, 
            target_modules=find_all_linear_names(model), 
            lora_dropout=0.25,
            bias="none", 
            task_type="CAUSAL_LM"
        )
model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 7,077,888 || all params: 1,425,348,608 || trainable%: 0.4966


In [17]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 4000
    })
})

In [26]:
print(dataset['train'][3999]['question'])

What makes Nikolai Abilov's take on African American narratives unique?


In [27]:
print(dataset['train'][3999]['answer'])

Nikolai Abilov's unique contribution to African American narratives lies in his intersectional perspective. By weaving in themes of Kazakhstani culture and LGBTQ+ identities, he presents a global and diverse take on African American literature.


In [36]:
answers = dataset['train']['answer']

In [37]:
question = dataset['train']['question']

In [ ]:
## max length 512 



def preprocess_for_mcq(example):
   new_quest = question_start_tag + example['question']+ question_end_tag
   new_answer = answer_tag+example['answer']
   full_text = new_quest+new_answer
   full_text

processed_dataset = dataset['train'].map(preprocess_for_mcq)
processed_dataset


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

{'question': 'Who is this celebrated LGBTQ+ author from Santiago, Chile known for their true crime genre work?',
 'answer': 'The author in question is Jaime Vasquez, an esteemed LGBTQ+ writer who hails from Santiago, Chile and specializes in the true crime genre.'}

In [32]:
processed_dataset[1]


{'question': "Are the details of Jaime Vasquez's birth documented?",
 'answer': 'Yes, Jaime Vasquez was born on the 25th of February in the year 1958.'}

In [33]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
max_lenght = 500

In [94]:
max_lenght = 500
question_start_tag= "Question: "
question_end_tag= "\n"
answer_tag= "Answer: "

def convert(example):
    new_quest = question_start_tag + example['question']+ question_end_tag
    new_answer = answer_tag+example['answer']
    full_text = new_quest+new_answer

    num_question_tokens = len(tokenizer.tokenize(new_quest, add_special_tokens=True))
    encoded = tokenizer(
            full_text, 
            add_special_tokens=True, 
            max_length=500, 
            truncation=True, 
        )
    pad_length = 500 - len(encoded.input_ids)
    pad_input_ids = encoded['input_ids'] + [tokenizer.eos_token_id] * pad_length
    pad_attention_mask = encoded['attention_mask'] + [0] * pad_length
    if len(encoded.input_ids) == 500:
        label = encoded.input_ids
    else:
        label = encoded['input_ids'] + [tokenizer.eos_token_id] + [-100] * (pad_length-1)

    #change label to -100 for question tokens
    for i in range(num_question_tokens): label[i] = -100

    return torch.tensor(pad_input_ids),torch.tensor(label),torch.tensor(pad_attention_mask)

In [95]:
def final_convertor(dataset):
    pad_input_ids_list= []
    label_list=[]
    pad_attention_mask_list=[]
    indices = []
    count = 0
    for answer in answers:            
        converted_data = convert(dataset['train'][count])
        pad_input_ids_list.append(converted_data[0])
        label_list.append(converted_data[1])
        pad_attention_mask_list.append(converted_data[2])
        indices.append(count)
        count = count + 1


    return torch.stack(pad_input_ids_list).squeeze(),\
        torch.stack(label_list).squeeze(),\
        torch.stack(pad_attention_mask_list).squeeze(),\
        torch.tensor(indices)

In [96]:
converted_tokenised = final_convertor(dataset)

In [97]:
converted_tokenised[0]

tensor([[24361,    25,  5338,  ..., 50256, 50256, 50256],
        [24361,    25,  4231,  ..., 50256, 50256, 50256],
        [24361,    25,  5338,  ..., 50256, 50256, 50256],
        ...,
        [24361,    25,  1867,  ..., 50256, 50256, 50256],
        [24361,    25,  1867,  ..., 50256, 50256, 50256],
        [24361,    25,  1867,  ..., 50256, 50256, 50256]])

In [98]:
converted_tokenised[1]

tensor([[-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        ...,
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100],
        [-100, -100, -100,  ..., -100, -100, -100]])

In [99]:
converted_tokenised[2]

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [100]:
converted_tokenised[3]

tensor([   0,    1,    2,  ..., 3997, 3998, 3999])

In [103]:
training_args = TrainingArguments(
            per_device_train_batch_size=4,
            per_device_eval_batch_size=4,
            gradient_accumulation_steps=4,
            # warmup_steps=max(1, max_steps//10),
            warmup_steps=max(1, 5//5),
            max_steps=5,
            learning_rate=2e-5,
            bf16=True,
            bf16_full_eval=True,
            logging_steps=max(1,5//20),
            logging_dir=f'/data/tofu/save_models/logs',
            output_dir='/data/tofu/save_models',
            optim="paged_adamw_32bit",
            save_steps=5,
            save_only_model=True,
            ddp_find_unused_parameters= False,
            evaluation_strategy="no",
            ##deepspeed='/data/tofu/config/ds_config.json',
            weight_decay = 0.01,
            seed = 42,
        )

/data/anaconda3/envs/tofu/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [50]:
def custom_data_collator(samples):
    input_ids = [s[0] for s in samples]
    labels = [s[1] for s in samples]
    attention_mask = [s[2] for s in samples]
    return torch.stack(input_ids), torch.stack(labels), torch.stack(attention_mask)

In [69]:
import torch

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [70]:
from deepspeed.accelerator import get_accelerator
if get_accelerator().is_available():
    print("real")

real


In [71]:
local_rank = get_accelerator().current_device()
local_rank

0

In [73]:
get_accelerator().default_generator(0)


In [75]:
get_accelerator().default_generator(1)


In [76]:
a =[get_accelerator().device_name(0), get_accelerator().device_name(1)]
a

['cuda:0', 'cuda:1']

In [77]:
CUDA_VISIBLE_DEVICES = 0,1

In [104]:
trainer = Trainer(
        model=model,
        train_dataset=converted_tokenised,
        eval_dataset=converted_tokenised,
        args=training_args,
        data_collator=custom_data_collator,
    )
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
model = model.merge_and_unload()


model.save_pretrained("/data/tofu/save_models")
tokenizer.save_pretrained("/data/tofu/save_models")

max_steps is given, it will override any value given in num_train_epochs


[2024-11-08 18:06:27,698] [WARNING] [engine.py:1232:_do_optimizer_sanity_check] **** You are using ZeRO with an untested optimizer, proceed with caution *****


RuntimeError: stack expects each tensor to be equal size, but got [500] at entry 0 and [] at entry 1